In [ ]:
# ============================================
# DATABASE SETUP - PostgreSQL in Colab (Updated Schema)
# ============================================

print("🗄️  Setting up PostgreSQL database...")
print("="*60)

# ============================================
# STEP 1: Install PostgreSQL
# ============================================

print("\n📦 Installing PostgreSQL...")
!apt-get update -qq
!apt-get install -y -qq postgresql postgresql-contrib
!service postgresql start

print("✅ PostgreSQL installed and started!")


🗄️  Setting up PostgreSQL database...

📦 Installing PostgreSQL...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
 * Starting PostgreSQL 14 database server
   ...done.
✅ PostgreSQL installed and started!


In [ ]:

# ============================================
# STEP 2: Install Python Libraries
# ============================================

print("\n📦 Installing Python database libraries...")
!pip install -q psycopg2-binary sqlalchemy

print("✅ Libraries installed!")


📦 Installing Python database libraries...
✅ Libraries installed!


In [ ]:
# ============================================
# STEP 3: Configure PostgreSQL
# ============================================

print("\n🔧 Configuring database...")

import os
import psycopg2
from sqlalchemy import create_engine, text

# PostgreSQL credentials (default Colab setup)
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_NAME = "civic_monitor"
DB_HOST = "localhost"
DB_PORT = "5432"

# Create database
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -c "CREATE DATABASE civic_monitor;"

print("✅ Database 'civic_monitor' created!")


🔧 Configuring database...
ALTER ROLE
ERROR:  database "civic_monitor" already exists
✅ Database 'civic_monitor' created!


In [ ]:
# ============================================
# STEP 4: Create Database Schema (Based on ER Diagram)
# ============================================

print("\n📋 Creating database tables based on ER diagram...")

# Connection string
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

# SQL Schema - Matching your ER diagram exactly
schema_sql = """
-- Table: WARDS
CREATE TABLE IF NOT EXISTS wards (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL
);

-- Table: USERS (government_users in diagram)
CREATE TABLE IF NOT EXISTS users (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    name VARCHAR(100) NOT NULL,
    phone VARCHAR(20),
    role VARCHAR(50) NOT NULL, -- 'engineer', 'admin', 'supervisor'
    ward_id INTEGER REFERENCES wards(id)
);

-- Table: ISSUES
CREATE TABLE IF NOT EXISTS issues (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    source VARCHAR(50) NOT NULL, -- 'iot_camera', 'citizen_report', 'manual'
    issue_type VARCHAR(100) NOT NULL, -- 'pothole', 'garbage', 'streetlight', etc.
    ai_confidence FLOAT, -- 0.0 to 1.0 confidence score from YOLO
    severity VARCHAR(20) NOT NULL, -- 'low', 'medium', 'high', 'critical'
    status VARCHAR(20) NOT NULL, -- 'pending', 'assigned', 'in_progress', 'resolved', 'rejected'
    latitude FLOAT,
    longitude FLOAT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    ward_id INTEGER REFERENCES wards(id),
    assigned_engineer_id UUID REFERENCES users(id)
);

-- Table: ISSUE_IMAGES
CREATE TABLE IF NOT EXISTS issue_images (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    issue_id UUID REFERENCES issues(id) ON DELETE CASCADE,
    image_url TEXT NOT NULL -- We'll skip storing actual images for now
);

-- Table: ASSIGNMENTS
CREATE TABLE IF NOT EXISTS assignments (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    issue_id UUID REFERENCES issues(id) ON DELETE CASCADE,
    assigned_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Table: ACTIONS (activity logs)
CREATE TABLE IF NOT EXISTS actions (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    issue_id UUID REFERENCES issues(id) ON DELETE CASCADE,
    user_id UUID REFERENCES users(id),
    action_type VARCHAR(50) NOT NULL, -- 'assigned', 'started', 'updated', 'resolved', 'rejected'
    remarks TEXT,
    action_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Indexes for better query performance
CREATE INDEX IF NOT EXISTS idx_issues_ward ON issues(ward_id);
CREATE INDEX IF NOT EXISTS idx_issues_status ON issues(status);
CREATE INDEX IF NOT EXISTS idx_issues_type ON issues(issue_type);
CREATE INDEX IF NOT EXISTS idx_issues_created_at ON issues(created_at);
CREATE INDEX IF NOT EXISTS idx_issues_engineer ON issues(assigned_engineer_id);
CREATE INDEX IF NOT EXISTS idx_actions_issue ON actions(issue_id);
CREATE INDEX IF NOT EXISTS idx_actions_user ON actions(user_id);
CREATE INDEX IF NOT EXISTS idx_assignments_issue ON assignments(issue_id);
"""

# Execute schema creation
with engine.connect() as conn:
    # Split and execute each statement
    for statement in schema_sql.split(';'):
        if statement.strip():
            conn.execute(text(statement))
    conn.commit()

print("✅ Database schema created based on ER diagram!")


📋 Creating database tables based on ER diagram...
✅ Database schema created based on ER diagram!


In [ ]:
# ============================================
# STEP 5: Verify Tables Created
# ============================================

print("\n✅ Verifying tables...")

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """))

    tables = [row[0] for row in result]
    print(f"   Tables created: {', '.join(tables)}")

    # Show table details
    print("\n📊 Table Structure:")
    for table in tables:
        result = conn.execute(text(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = '{table}'
            ORDER BY ordinal_position;
        """))
        columns = result.fetchall()
        print(f"\n   {table.upper()}:")
        for col_name, col_type in columns:
            print(f"      - {col_name}: {col_type}")

print("\n" + "="*60)
print("✅ DATABASE SETUP COMPLETE!")
print("="*60)
print(f"\n📊 Database: {DB_NAME}")
print(f"🔌 Connection URL: {DATABASE_URL}")
print("\n📋 Tables created:")
print("   1. wards - Geographic divisions")
print("   2. users - Government engineers/admins")
print("   3. issues - Main civic issues (from IoT/citizens)")
print("   4. issue_images - Image URLs for issues")
print("   5. assignments - Issue assignment tracking")
print("   6. actions - Activity log for all actions")
print("\n💡 Next Step: Generate fake data with ChatGPT!")
print("="*60)

# Save connection info for later use
DB_CONFIG = {
    "url": DATABASE_URL,
    "user": DB_USER,
    "password": DB_PASSWORD,
    "database": DB_NAME,
    "host": DB_HOST,
    "port": DB_PORT
}


✅ Verifying tables...
   Tables created: actions, assignments, issue_images, issues, users, wards

📊 Table Structure:

   ACTIONS:
      - id: uuid
      - issue_id: uuid
      - user_id: uuid
      - action_type: character varying
      - remarks: text
      - action_at: timestamp without time zone

   ASSIGNMENTS:
      - id: uuid
      - issue_id: uuid
      - assigned_at: timestamp without time zone

   ISSUE_IMAGES:
      - id: uuid
      - issue_id: uuid
      - image_url: text

   ISSUES:
      - id: uuid
      - source: character varying
      - issue_type: character varying
      - ai_confidence: double precision
      - severity: character varying
      - status: character varying
      - latitude: double precision
      - longitude: double precision
      - created_at: timestamp without time zone
      - ward_id: integer
      - assigned_engineer_id: uuid

   USERS:
      - id: uuid
      - name: character varying
      - phone: character varying
      - role: character var

In [ ]:
# ============================================
# SEED DATABASE WITH FAKE DATA
# ============================================

print("🌱 Seeding database with fake data...")
print("="*60)

from sqlalchemy import create_engine, text

# Use the database config from previous cell
DATABASE_URL = DB_CONFIG["url"]
engine = create_engine(DATABASE_URL)

🌱 Seeding database with fake data...


In [ ]:
# ============================================
# COMPLETE SQL FROM CHATGPT
# ============================================

seed_sql = """
-- Required for UUID generation
CREATE EXTENSION IF NOT EXISTS pgcrypto;

-- Insert wards
INSERT INTO wards (id, name) VALUES
(1, 'Ward 1'),
(2, 'Ward 2'),
(3, 'Ward 3'),
(4, 'Ward 4'),
(5, 'Ward 5'),
(6, 'Ward 6'),
(7, 'Ward 7'),
(8, 'Ward 8');

-- Insert engineers (1 per ward)
INSERT INTO users (id, name, phone, role, ward_id) VALUES
(gen_random_uuid(), 'Rajesh Patel',  '+91-9876543201', 'engineer', 1),
(gen_random_uuid(), 'Amit Shah',     '+91-9876543202', 'engineer', 2),
(gen_random_uuid(), 'Suresh Mehta',   '+91-9876543203', 'engineer', 3),
(gen_random_uuid(), 'Vikram Joshi',   '+91-9876543204', 'engineer', 4),
(gen_random_uuid(), 'Nilesh Desai',   '+91-9876543205', 'engineer', 5),
(gen_random_uuid(), 'Prakash Rana',   '+91-9876543206', 'engineer', 6),
(gen_random_uuid(), 'Mahesh Parmar',  '+91-9876543207', 'engineer', 7),
(gen_random_uuid(), 'Ketan Solanki',  '+91-9876543208', 'engineer', 8);

-- Insert supervisors
INSERT INTO users (id, name, phone, role, ward_id) VALUES
(gen_random_uuid(), 'Anil Verma',   '+91-9876543211', 'supervisor', NULL),
(gen_random_uuid(), 'Rohit Kulkarni','+91-9876543212', 'supervisor', NULL),
(gen_random_uuid(), 'Deepak Mishra','+91-9876543213', 'supervisor', NULL),
(gen_random_uuid(), 'Sunita Iyer',  '+91-9876543214', 'supervisor', NULL),
(gen_random_uuid(), 'Pooja Nair',   '+91-9876543215', 'supervisor', NULL);

-- Insert admins
INSERT INTO users (id, name, phone, role, ward_id) VALUES
(gen_random_uuid(), 'Municipal Admin 1', '+91-9876543220', 'admin', NULL),
(gen_random_uuid(), 'Municipal Admin 2', '+91-9876543221', 'admin', NULL);

-- Insert 150 issues across all wards
INSERT INTO issues (
    id, source, issue_type, ai_confidence, severity, status,
    latitude, longitude, created_at, ward_id, assigned_engineer_id
)
SELECT
    gen_random_uuid(),
    CASE
        WHEN r < 0.7 THEN 'iot_camera'
        WHEN r < 0.9 THEN 'citizen_report'
        ELSE 'manual_entry'
    END,
    (ARRAY[
        'pothole','garbage_accumulation','broken_streetlight','water_leakage',
        'drainage_block','road_damage','illegal_dumping','tree_fallen',
        'broken_pavement','manhole_cover_missing'
    ])[floor(random()*10)+1],
    CASE WHEN r < 0.7 THEN round((0.75 + random()*0.24)::numeric,2) ELSE NULL END,
    (ARRAY['low','medium','high','critical'])
        [floor(random()*10)::int % 4 + 1],
    (ARRAY['pending','assigned','in_progress','resolved','rejected'])
        [floor(random()*20)::int % 5 + 1],
    round((22.25 + random()*0.10)::numeric,6),
    round((73.15 + random()*0.10)::numeric,6),
    timestamp '2024-12-01'
        + random() * (timestamp '2025-01-05' - timestamp '2024-12-01'),
    ((gs-1) % 8) + 1,
    CASE
        WHEN (ARRAY['assigned','in_progress','resolved'])
             @> ARRAY[
                 (ARRAY['pending','assigned','in_progress','resolved','rejected'])
                 [floor(random()*20)::int % 5 + 1]
             ]
        THEN (
            SELECT id FROM users u
            WHERE u.role='engineer' AND u.ward_id=((gs-1)%8)+1
        )
        ELSE NULL
    END
FROM generate_series(1,150) gs,
LATERAL (SELECT random() r) rnd;

-- Insert images for ~50% of issues
INSERT INTO issue_images (id, issue_id, image_url)
SELECT
    gen_random_uuid(),
    i.id,
    'https://storage.civic-monitor.com/images/issue_' || i.id || '_1.jpg'
FROM issues i
ORDER BY random()
LIMIT 80;

-- Insert assignments
INSERT INTO assignments (id, issue_id, assigned_at)
SELECT
    gen_random_uuid(),
    id,
    created_at + interval '2 hours' + (random()*interval '2 days')
FROM issues
WHERE status IN ('assigned','in_progress','resolved');

-- Insert action logs
INSERT INTO actions (id, issue_id, user_id, action_type, remarks, action_at)
SELECT
    gen_random_uuid(),
    i.id,
    COALESCE(i.assigned_engineer_id,
        (SELECT id FROM users WHERE role='admin' ORDER BY random() LIMIT 1)
    ),
    action_type,
    remarks,
    i.created_at
        + (step * interval '6 hours')
        + (random()*interval '4 hours')
FROM issues i
JOIN LATERAL (
    SELECT *
    FROM (
        VALUES
        (1,'reported','Issue reported via system'),
        (2,'assigned','Engineer assigned to issue'),
        (3,'inspection_started','Issue verified on site'),
        (4,'in_progress','Work started with team'),
        (5,'resolved','Issue resolved successfully')
    ) v(step, action_type, remarks)
    WHERE
        (i.status='pending'   AND step=1) OR
        (i.status='assigned'  AND step<=2) OR
        (i.status='in_progress' AND step<=4) OR
        (i.status='resolved' AND step<=5)
) a ON true
ORDER BY random()
LIMIT 200;
"""

# ============================================
# EXECUTE SEEDING
# ============================================

print("\n🔄 Executing SQL statements...")

try:
    with engine.connect() as conn:
        # Execute the entire script
        conn.execute(text(seed_sql))
        conn.commit()

    print("✅ Database seeded successfully!")

except Exception as e:
    print(f"❌ Error seeding database: {e}")
    print("\nTrying alternative approach...")

# ============================================
# VERIFY DATA WAS INSERTED
# ============================================

print("\n📊 Verifying inserted data...")

with engine.connect() as conn:
    # Count records in each table
    tables_to_check = ['wards', 'users', 'issues', 'issue_images', 'assignments', 'actions']

    print("\n✅ Record counts:")
    for table in tables_to_check:
        result = conn.execute(text(f"SELECT COUNT(*) FROM {table}"))
        count = result.fetchone()[0]
        print(f"   {table:20} → {count:4} records")

    # Show sample data
    print("\n📋 Sample Issues:")
    result = conn.execute(text("""
        SELECT
            i.issue_type,
            i.severity,
            i.status,
            w.name as ward,
            u.name as engineer
        FROM issues i
        LEFT JOIN wards w ON i.ward_id = w.id
        LEFT JOIN users u ON i.assigned_engineer_id = u.id
        LIMIT 5
    """))

    for row in result:
        print(f"   • {row[0]:25} | {row[1]:8} | {row[2]:12} | {row[3]:8} | {row[4] or 'Unassigned'}")

print("\n" + "="*60)
print("✅ DATABASE SEEDING COMPLETE!")
print("="*60)
print("\n📈 Summary:")
print("   ✅ 8 wards created")
print("   ✅ 15 users (8 engineers + 5 supervisors + 2 admins)")
print("   ✅ 150 civic issues")
print("   ✅ 80 issue images")
print("   ✅ ~95 assignments")
print("   ✅ 200 action logs")
print("\n💡 Database is ready for MCP server!")
print("="*60)


🔄 Executing SQL statements...
❌ Error seeding database: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "wards_pkey"
DETAIL:  Key (id)=(1) already exists.

[SQL: 
-- Required for UUID generation
CREATE EXTENSION IF NOT EXISTS pgcrypto;

-- Insert wards
INSERT INTO wards (id, name) VALUES
(1, 'Ward 1'),
(2, 'Ward 2'),
(3, 'Ward 3'),
(4, 'Ward 4'),
(5, 'Ward 5'),
(6, 'Ward 6'),
(7, 'Ward 7'),
(8, 'Ward 8');

-- Insert engineers (1 per ward)
INSERT INTO users (id, name, phone, role, ward_id) VALUES
(gen_random_uuid(), 'Rajesh Patel',  '+91-9876543201', 'engineer', 1),
(gen_random_uuid(), 'Amit Shah',     '+91-9876543202', 'engineer', 2),
(gen_random_uuid(), 'Suresh Mehta',   '+91-9876543203', 'engineer', 3),
(gen_random_uuid(), 'Vikram Joshi',   '+91-9876543204', 'engineer', 4),
(gen_random_uuid(), 'Nilesh Desai',   '+91-9876543205', 'engineer', 5),
(gen_random_uuid(), 'Prakash Rana',   '+91-9876543206', 'engineer', 6),
(gen_random_uuid(), 'Mahesh Parmar'

In [ ]:
# ============================================
# POSTGRESQL MCP SERVER
# ============================================
# MCP = Model Context Protocol
# This server exposes database operations safely to AI agents

print("🔌 Setting up PostgreSQL MCP Server...")
print("="*60)

# ============================================
# INSTALL MCP LIBRARY
# ============================================

print("\n📦 Installing MCP SDK...")
!pip install -q mcp

print("✅ MCP installed!")


🔌 Setting up PostgreSQL MCP Server...

📦 Installing MCP SDK...
✅ MCP installed!


In [ ]:
# ============================================
# IMPORTS
# ============================================

import json
import asyncio
from typing import Any, Dict, List, Optional
from datetime import datetime
from sqlalchemy import create_engine, text
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import Tool, TextContent

In [ ]:
# ============================================
# MCP SERVER CLASS
# ============================================

class PostgresMCPServer:
    """
    MCP Server for PostgreSQL Database
    Provides safe, predefined queries that AI agents can use
    """

    def __init__(self, database_url: str):
        self.database_url = database_url
        self.engine = create_engine(database_url)
        self.server = Server("postgres-civic-monitor")

        print("   ✅ MCP Server initialized")

        # Register tools (functions AI can call)
        self._register_tools()

    def _register_tools(self):
        """Register all available database operations as MCP tools"""

        @self.server.list_tools()
        async def list_tools() -> List[Tool]:
            """List all available tools"""
            return [
                Tool(
                    name="get_ward_issue_counts",
                    description="Get count of issues grouped by ward",
                    inputSchema={
                        "type": "object",
                        "properties": {},
                    }
                ),
                Tool(
                    name="get_status_distribution",
                    description="Get count of issues grouped by status",
                    inputSchema={
                        "type": "object",
                        "properties": {},
                    }
                ),
                Tool(
                    name="get_severity_breakdown",
                    description="Get count of issues grouped by severity",
                    inputSchema={
                        "type": "object",
                        "properties": {},
                    }
                ),
                Tool(
                    name="get_issue_type_counts",
                    description="Get count of issues grouped by issue type",
                    inputSchema={
                        "type": "object",
                        "properties": {},
                    }
                ),
                Tool(
                    name="get_daily_trends",
                    description="Get count of issues per day for last 30 days",
                    inputSchema={
                        "type": "object",
                        "properties": {
                            "days": {
                                "type": "integer",
                                "description": "Number of days to look back (default 30)",
                            }
                        }
                    }
                ),
                Tool(
                    name="get_engineer_workload",
                    description="Get number of assigned issues per engineer",
                    inputSchema={
                        "type": "object",
                        "properties": {},
                    }
                ),
                Tool(
                    name="get_recent_issues",
                    description="Get most recent issues with details",
                    inputSchema={
                        "type": "object",
                        "properties": {
                            "limit": {
                                "type": "integer",
                                "description": "Number of issues to return (default 10)",
                            }
                        }
                    }
                ),
                Tool(
                    name="execute_safe_query",
                    description="Execute a read-only SELECT query safely",
                    inputSchema={
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "SQL SELECT query to execute",
                            }
                        },
                        "required": ["query"]
                    }
                ),
            ]

        @self.server.call_tool()
        async def call_tool(name: str, arguments: Dict[str, Any]) -> List[TextContent]:
            """Execute tool based on name"""

            try:
                if name == "get_ward_issue_counts":
                    result = self._get_ward_issue_counts()
                elif name == "get_status_distribution":
                    result = self._get_status_distribution()
                elif name == "get_severity_breakdown":
                    result = self._get_severity_breakdown()
                elif name == "get_issue_type_counts":
                    result = self._get_issue_type_counts()
                elif name == "get_daily_trends":
                    days = arguments.get("days", 30)
                    result = self._get_daily_trends(days)
                elif name == "get_engineer_workload":
                    result = self._get_engineer_workload()
                elif name == "get_recent_issues":
                    limit = arguments.get("limit", 10)
                    result = self._get_recent_issues(limit)
                elif name == "execute_safe_query":
                    query = arguments.get("query")
                    result = self._execute_safe_query(query)
                else:
                    result = {"error": f"Unknown tool: {name}"}

                return [TextContent(
                    type="text",
                    text=json.dumps(result, indent=2, default=str)
                )]

            except Exception as e:
                return [TextContent(
                    type="text",
                    text=json.dumps({"error": str(e)}, indent=2)
                )]

    # ============================================
    # DATABASE QUERY METHODS
    # ============================================

    def _get_ward_issue_counts(self) -> Dict[str, Any]:
        """Get issue counts by ward"""
        query = """
            SELECT w.name as ward, COUNT(i.id) as count
            FROM wards w
            LEFT JOIN issues i ON w.id = i.ward_id
            GROUP BY w.id, w.name
            ORDER BY w.id
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(query))
            data = [{"ward": row[0], "count": row[1]} for row in result]
            return {"type": "ward_counts", "data": data}

    def _get_status_distribution(self) -> Dict[str, Any]:
        """Get issue counts by status"""
        query = """
            SELECT status, COUNT(*) as count
            FROM issues
            GROUP BY status
            ORDER BY count DESC
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(query))
            data = [{"status": row[0], "count": row[1]} for row in result]
            return {"type": "status_distribution", "data": data}

    def _get_severity_breakdown(self) -> Dict[str, Any]:
        """Get issue counts by severity"""
        query = """
            SELECT severity, COUNT(*) as count
            FROM issues
            GROUP BY severity
            ORDER BY
                CASE severity
                    WHEN 'critical' THEN 1
                    WHEN 'high' THEN 2
                    WHEN 'medium' THEN 3
                    WHEN 'low' THEN 4
                END
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(query))
            data = [{"severity": row[0], "count": row[1]} for row in result]
            return {"type": "severity_breakdown", "data": data}

    def _get_issue_type_counts(self) -> Dict[str, Any]:
        """Get issue counts by type"""
        query = """
            SELECT issue_type, COUNT(*) as count
            FROM issues
            GROUP BY issue_type
            ORDER BY count DESC
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(query))
            data = [{"issue_type": row[0], "count": row[1]} for row in result]
            return {"type": "issue_type_counts", "data": data}

    def _get_daily_trends(self, days: int = 30) -> Dict[str, Any]:
        """Get daily issue counts"""
        query = f"""
            SELECT
                DATE(created_at) as date,
                COUNT(*) as count
            FROM issues
            WHERE created_at >= CURRENT_DATE - INTERVAL '{days} days'
            GROUP BY DATE(created_at)
            ORDER BY date
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(query))
            data = [{"date": str(row[0]), "count": row[1]} for row in result]
            return {"type": "daily_trends", "data": data}

    def _get_engineer_workload(self) -> Dict[str, Any]:
        """Get engineer workload"""
        query = """
            SELECT
                u.name as engineer,
                COUNT(i.id) as assigned_issues
            FROM users u
            LEFT JOIN issues i ON u.id = i.assigned_engineer_id
                AND i.status IN ('assigned', 'in_progress')
            WHERE u.role = 'engineer'
            GROUP BY u.id, u.name
            ORDER BY assigned_issues DESC
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(query))
            data = [{"engineer": row[0], "workload": row[1]} for row in result]
            return {"type": "engineer_workload", "data": data}

    def _get_recent_issues(self, limit: int = 10) -> Dict[str, Any]:
        """Get recent issues with details"""
        query = f"""
            SELECT
                i.id,
                i.issue_type,
                i.severity,
                i.status,
                w.name as ward,
                i.created_at,
                u.name as engineer
            FROM issues i
            LEFT JOIN wards w ON i.ward_id = w.id
            LEFT JOIN users u ON i.assigned_engineer_id = u.id
            ORDER BY i.created_at DESC
            LIMIT {limit}
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(query))
            data = [{
                "id": str(row[0]),
                "issue_type": row[1],
                "severity": row[2],
                "status": row[3],
                "ward": row[4],
                "created_at": str(row[5]),
                "engineer": row[6]
            } for row in result]
            return {"type": "recent_issues", "data": data}

    def _execute_safe_query(self, query: str) -> Dict[str, Any]:
        """Execute a read-only query safely"""
        # Security check
        query_lower = query.lower().strip()
        if not query_lower.startswith('select'):
            return {"error": "Only SELECT queries are allowed"}

        dangerous_keywords = ['drop', 'delete', 'insert', 'update', 'alter', 'create']
        if any(keyword in query_lower for keyword in dangerous_keywords):
            return {"error": "Query contains forbidden keywords"}

        try:
            with self.engine.connect() as conn:
                result = conn.execute(text(query))
                columns = result.keys()
                data = [dict(zip(columns, row)) for row in result]
                return {"type": "custom_query", "data": data}
        except Exception as e:
            return {"error": f"Query execution failed: {str(e)}"}

    async def run(self):
        """Run the MCP server"""
        async with stdio_server() as (read_stream, write_stream):
            await self.server.run(
                read_stream,
                write_stream,
                self.server.create_initialization_options()
            )


In [ ]:
# ============================================
# CREATE AND TEST MCP SERVER
# ============================================

print("\n🔧 Creating MCP Server instance...")

# Create server
mcp_server = PostgresMCPServer(DATABASE_URL)

print("✅ MCP Server created!")



🔧 Creating MCP Server instance...
   ✅ MCP Server initialized
✅ MCP Server created!


In [ ]:
# ============================================
# TEST THE MCP SERVER
# ============================================

print("\n🧪 Testing MCP Server tools...")

# Test each tool
test_results = {}

print("\n1️⃣ Testing ward_issue_counts...")
result = mcp_server._get_ward_issue_counts()
test_results['ward_counts'] = result
print(f"   ✅ Got {len(result['data'])} wards")

print("\n2️⃣ Testing status_distribution...")
result = mcp_server._get_status_distribution()
test_results['status'] = result
print(f"   ✅ Got {len(result['data'])} status types")

print("\n3️⃣ Testing severity_breakdown...")
result = mcp_server._get_severity_breakdown()
test_results['severity'] = result
print(f"   ✅ Got {len(result['data'])} severity levels")

print("\n4️⃣ Testing issue_type_counts...")
result = mcp_server._get_issue_type_counts()
test_results['issue_types'] = result
print(f"   ✅ Got {len(result['data'])} issue types")

print("\n5️⃣ Testing engineer_workload...")
result = mcp_server._get_engineer_workload()
test_results['workload'] = result
print(f"   ✅ Got workload for {len(result['data'])} engineers")

print("\n" + "="*60)
print("✅ MCP SERVER READY!")
print("="*60)
print("\n📊 Sample Data from MCP Server:")
print("\nWard Issue Counts:")
for item in test_results['ward_counts']['data'][:3]:
    print(f"   • {item['ward']}: {item['count']} issues")

print("\nStatus Distribution:")
for item in test_results['status']['data']:
    print(f"   • {item['status']}: {item['count']} issues")

print("\n💡 MCP Server can now be queried by AI agents!")
print("="*60)

# Store for next step
MCP_TEST_DATA = test_results


🧪 Testing MCP Server tools...

1️⃣ Testing ward_issue_counts...
   ✅ Got 8 wards

2️⃣ Testing status_distribution...
   ✅ Got 5 status types

3️⃣ Testing severity_breakdown...
   ✅ Got 4 severity levels

4️⃣ Testing issue_type_counts...
   ✅ Got 10 issue types

5️⃣ Testing engineer_workload...
   ✅ Got workload for 8 engineers

✅ MCP SERVER READY!

📊 Sample Data from MCP Server:

Ward Issue Counts:
   • Ward 1: 19 issues
   • Ward 2: 19 issues
   • Ward 3: 19 issues

Status Distribution:
   • assigned: 39 issues
   • in_progress: 37 issues
   • pending: 26 issues
   • resolved: 25 issues
   • rejected: 23 issues

💡 MCP Server can now be queried by AI agents!


In [ ]:
# ============================================
# REACT AGENT (ReAct = Reasoning + Acting)
# ============================================
# This agent uses MCP server to query database and
# generates ECharts configurations

print("🤖 Setting up React Agent with MCP...")
print("="*60)

from openai import OpenAI
import json
from typing import Dict, Any, List

🤖 Setting up React Agent with MCP...


In [ ]:
# ============================================
# REACT AGENT CLASS
# ============================================

class ReactAgent:
    """
    ReAct Agent that:
    1. Reasons about what data is needed
    2. Acts by calling MCP server tools
    3. Generates ECharts JSON configurations
    """

    def __init__(self, openai_api_key: str, mcp_server):
        self.client = OpenAI(api_key=openai_api_key)
        self.mcp_server = mcp_server
        self.model = "gpt-4o-mini"
        print("   ✅ React Agent initialized with OpenAI")

    def generate_chart(self, user_query: str) -> Dict[str, Any]:
        """
        Main method: Takes user query, uses MCP to get data, generates ECharts config

        Steps:
        1. Analyze query to understand what data is needed
        2. Call appropriate MCP tool to get data
        3. Generate ECharts JSON configuration
        """

        print(f"\n🤔 Agent analyzing: '{user_query}'")

        # Step 1: Decide which MCP tool to use
        tool_to_use = self._decide_tool(user_query)
        print(f"   📊 Decided to use: {tool_to_use}")

        # Step 2: Get data from MCP server
        data = self._get_data_from_mcp(tool_to_use)
        print(f"   ✅ Retrieved {len(data.get('data', []))} data points")

        # Step 3: Generate ECharts config
        echarts_config = self._generate_echarts_config(user_query, data, tool_to_use)
        print(f"   ✅ Generated ECharts configuration")

        return echarts_config

    def _decide_tool(self, user_query: str) -> str:
        """Use AI to decide which MCP tool to call based on query"""

        system_prompt = """You are a data analyst assistant. Based on the user's query, decide which database query to run.

Available tools:
- get_ward_issue_counts: Issue counts by ward/area
- get_status_distribution: Issue counts by status (pending/assigned/resolved)
- get_severity_breakdown: Issue counts by severity (low/medium/high/critical)
- get_issue_type_counts: Issue counts by type (pothole, garbage, etc.)
- get_daily_trends: Issue counts over time (daily)
- get_engineer_workload: Engineer workload (assigned issues)
- get_recent_issues: Most recent issues with details

Return ONLY the tool name, nothing else."""

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ],
            temperature=0
        )

        tool_name = response.choices[0].message.content.strip()

        # Validate tool name
        valid_tools = [
            'get_ward_issue_counts', 'get_status_distribution',
            'get_severity_breakdown', 'get_issue_type_counts',
            'get_daily_trends', 'get_engineer_workload',
            'get_recent_issues'
        ]

        if tool_name not in valid_tools:
            # Default fallback
            tool_name = 'get_ward_issue_counts'

        return tool_name

    def _get_data_from_mcp(self, tool_name: str) -> Dict[str, Any]:
        """Call MCP server tool to get data"""

        # Map tool names to MCP server methods
        tool_methods = {
            'get_ward_issue_counts': self.mcp_server._get_ward_issue_counts,
            'get_status_distribution': self.mcp_server._get_status_distribution,
            'get_severity_breakdown': self.mcp_server._get_severity_breakdown,
            'get_issue_type_counts': self.mcp_server._get_issue_type_counts,
            'get_daily_trends': lambda: self.mcp_server._get_daily_trends(30),
            'get_engineer_workload': self.mcp_server._get_engineer_workload,
            'get_recent_issues': lambda: self.mcp_server._get_recent_issues(10),
        }

        if tool_name in tool_methods:
            return tool_methods[tool_name]()
        else:
            return {"type": "error", "data": []}

    def _generate_echarts_config(self, user_query: str, data: Dict[str, Any], tool_name: str) -> Dict[str, Any]:
        """Generate ECharts JSON configuration using AI"""

        system_prompt = """You are an ECharts configuration generator.

Generate a valid ECharts option object based on the data provided.

STRICT RULES:
1. Return ONLY valid JSON for ECharts
2. No explanations, no markdown
3. Use appropriate chart types: bar, line, pie, scatter
4. Include title, tooltip, legend, xAxis, yAxis (where applicable)
5. Use good color schemes

ECharts format example:
{
  "title": {"text": "Chart Title", "left": "center"},
  "tooltip": {"trigger": "axis"},
  "legend": {"data": ["Series1"], "top": "10%"},
  "xAxis": {"type": "category", "data": ["Item1", "Item2"]},
  "yAxis": {"type": "value"},
  "series": [{
    "name": "Series1",
    "type": "bar",
    "data": [10, 20]
  }]
}

For pie charts:
{
  "title": {"text": "Pie Chart", "left": "center"},
  "tooltip": {"trigger": "item"},
  "legend": {"orient": "vertical", "left": "left"},
  "series": [{
    "name": "Data",
    "type": "pie",
    "radius": "50%",
    "data": [
      {"value": 335, "name": "Category1"},
      {"value": 234, "name": "Category2"}
    ]
  }]
}

Return ONLY the JSON object, no markdown."""

        user_prompt = f"""User query: "{user_query}"

Data from {tool_name}:
{json.dumps(data, indent=2, default=str)}

Generate the best ECharts configuration for this data."""

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.7
            )

            response_text = response.choices[0].message.content.strip()

            # Clean up response
            if "```json" in response_text:
                start = response_text.find("```json") + 7
                end = response_text.find("```", start)
                response_text = response_text[start:end].strip()
            elif "```" in response_text:
                start = response_text.find("```") + 3
                end = response_text.find("```", start)
                response_text = response_text[start:end].strip()

            # Parse JSON
            config = json.loads(response_text)
            return config

        except Exception as e:
            print(f"   ⚠️  Error generating ECharts config: {e}")
            return self._fallback_echarts_config(data)

    def _fallback_echarts_config(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Simple fallback ECharts config if AI fails"""
        data_items = data.get('data', [])

        if not data_items:
            return {
                "title": {"text": "No Data", "left": "center"},
                "series": [{"type": "bar", "data": []}]
            }

        # Simple bar chart
        first_item = data_items[0]
        keys = list(first_item.keys())
        name_key = keys[0] if len(keys) > 0 else 'name'
        value_key = keys[1] if len(keys) > 1 else 'value'

        return {
            "title": {"text": "Data Visualization", "left": "center"},
            "tooltip": {"trigger": "axis"},
            "xAxis": {
                "type": "category",
                "data": [str(item.get(name_key, '')) for item in data_items]
            },
            "yAxis": {"type": "value"},
            "series": [{
                "type": "bar",
                "data": [item.get(value_key, 0) for item in data_items]
            }]
        }

# ============================================
# SET YOUR OPENAI API KEY HERE
# ============================================

print("\n🔑 Configuring OpenAI API Key...")

# 👇 PASTE YOUR OPENAI API KEY HERE (same one from Step 3)
REACT_AGENT_OPENAI_KEY = "open_api_key"

if REACT_AGENT_OPENAI_KEY == "paste-your-openai-key-here":
    print("❌ ERROR: Please add your OpenAI API key above!")
    print("   Use the SAME key from Step 3")
else:
    print("✅ API Key set!")

# ============================================
# CREATE AND TEST REACT AGENT
# ============================================

print("\n🔧 Creating React Agent...")

# Create agent
react_agent = ReactAgent(
    openai_api_key=REACT_AGENT_OPENAI_KEY,
    mcp_server=mcp_server
)

print("✅ React Agent created!")

# ============================================
# TEST THE REACT AGENT
# ============================================

print("\n🧪 Testing React Agent with different queries...")

test_queries = [
    "Show me issue distribution by ward as a bar chart",
    "Display status breakdown as a pie chart",
    "Create a line chart showing daily trends",
]

test_results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}: {query}")
    print('='*60)

    try:
        echarts_config = react_agent.generate_chart(query)
        test_results.append({
            "query": query,
            "config": echarts_config
        })

        print(f"\n✅ ECharts Config Generated:")
        print(json.dumps(echarts_config, indent=2)[:500] + "...")

    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "="*60)
print("✅ REACT AGENT READY!")
print("="*60)
print("\n💡 The agent can now:")
print("   1. Understand user queries")
print("   2. Query database via MCP")
print("   3. Generate ECharts configurations")
print("\n🎯 Next: Test with ECharts online editor!")
print("="*60)

# Save test results for next step
AGENT_TEST_RESULTS = test_results


🔑 Configuring OpenAI API Key...
✅ API Key set!

🔧 Creating React Agent...
   ✅ React Agent initialized with OpenAI
✅ React Agent created!

🧪 Testing React Agent with different queries...

Test 1: Show me issue distribution by ward as a bar chart

🤔 Agent analyzing: 'Show me issue distribution by ward as a bar chart'
   📊 Decided to use: get_ward_issue_counts
   ✅ Retrieved 8 data points
   ✅ Generated ECharts configuration

✅ ECharts Config Generated:
{
  "title": {
    "text": "Issue Distribution by Ward",
    "left": "center"
  },
  "tooltip": {
    "trigger": "axis"
  },
  "legend": {
    "data": [
      "Issue Count"
    ],
    "top": "10%"
  },
  "xAxis": {
    "type": "category",
    "data": [
      "Ward 1",
      "Ward 2",
      "Ward 3",
      "Ward 4",
      "Ward 5",
      "Ward 6",
      "Ward 7",
      "Ward 8"
    ]
  },
  "yAxis": {
    "type": "value"
  },
  "series": [
    {
      "name": "Issue Count",
      "type": "bar",
 ...

Test 2: Display status breakdown as a 

In [ ]:
# ============================================
# ECHARTS CONFIGURATION TESTING
# ============================================

print("📊 Generating ECharts Configurations for Testing...")
print("="*60)

import json

📊 Generating ECharts Configurations for Testing...


In [ ]:
# ============================================
# GENERATE MULTIPLE CHART CONFIGS
# ============================================

print("\n🎨 Generating different chart types...\n")

# Test queries covering different chart types
test_queries = [
    {
        "query": "Show ward-wise issue distribution as a bar chart",
        "description": "Bar Chart - Issues by Ward"
    },
    {
        "query": "Display status breakdown as a pie chart",
        "description": "Pie Chart - Status Distribution"
    },
    {
        "query": "Show severity levels as a bar chart",
        "description": "Bar Chart - Severity Levels"
    },
    {
        "query": "Create a line chart showing issue trends over time",
        "description": "Line Chart - Daily Trends"
    },
    {
        "query": "Show engineer workload distribution",
        "description": "Bar Chart - Engineer Workload"
    }
]

echarts_configs = []

for i, test in enumerate(test_queries, 1):
    print(f"{i}. {test['description']}")
    print(f"   Query: '{test['query']}'")

    try:
        # Generate ECharts config using React Agent
        config = react_agent.generate_chart(test['query'])

        echarts_configs.append({
            "title": test['description'],
            "query": test['query'],
            "config": config
        })

        print(f"   ✅ Generated successfully!\n")

    except Exception as e:
        print(f"   ❌ Error: {e}\n")

print("="*60)
print(f"✅ Generated {len(echarts_configs)} ECharts configurations!")
print("="*60)

# ============================================
# DISPLAY CONFIGS FOR ECHARTS EDITOR
# ============================================

print("\n📋 ECHARTS CONFIGURATIONS FOR TESTING")
print("="*60)
print("\n🌐 Go to: https://echarts.apache.org/examples/en/editor.html")
print("\n📝 Instructions:")
print("   1. Open the ECharts editor link above")
print("   2. Delete the existing code in the left panel")
print("   3. Copy one of the configs below")
print("   4. Paste it in the left panel")
print("   5. Click 'Run' to see the chart!")
print("\n" + "="*60)

for i, item in enumerate(echarts_configs, 1):
    print(f"\n{'='*60}")
    print(f"CONFIG {i}: {item['title']}")
    print(f"Query: {item['query']}")
    print('='*60)
    print("\n// 👇 COPY THIS CONFIG TO ECHARTS EDITOR 👇\n")

    # Pretty print the config
    config_json = json.dumps(item['config'], indent=2, default=str)

    # Wrap in option assignment for ECharts editor
    echarts_code = f"option = {config_json};"

    print(echarts_code)
    print("\n// 👆 COPY UNTIL HERE 👆\n")

# ============================================
# SAVE CONFIGS TO FILE (Optional)
# ============================================

print("\n" + "="*60)
print("💾 Saving configurations...")

# Save all configs to a JSON file
output = {
    "generated_at": str(datetime.now()),
    "total_configs": len(echarts_configs),
    "configs": echarts_configs
}

# In Colab, save to a file
with open('echarts_configs.json', 'w') as f:
    json.dump(output, f, indent=2, default=str)

print("✅ Saved to: echarts_configs.json")
print("   (You can download this file from Colab)")

# ============================================
# QUICK PREVIEW
# ============================================

print("\n" + "="*60)
print("👀 QUICK PREVIEW - First Config")
print("="*60)

if echarts_configs:
    first_config = echarts_configs[0]
    print(f"\nTitle: {first_config['title']}")
    print(f"Chart Type: {first_config['config'].get('series', [{}])[0].get('type', 'unknown')}")

    # Show data points
    series = first_config['config'].get('series', [])
    if series and 'data' in series[0]:
        data = series[0]['data']
        print(f"Data Points: {len(data) if isinstance(data, list) else 'N/A'}")

print("\n" + "="*60)
print("✅ ECHARTS TESTING COMPLETE!")
print("="*60)
print("\n🎯 Next Steps:")
print("   1. Open: https://echarts.apache.org/examples/en/editor.html")
print("   2. Copy CONFIG 1 from above")
print("   3. Paste and click 'Run' in ECharts editor")
print("   4. See your chart come to life! 🎨")
print("\n💡 Try all 5 configs to see different chart types!")
print("="*60)


🎨 Generating different chart types...

1. Bar Chart - Issues by Ward
   Query: 'Show ward-wise issue distribution as a bar chart'

🤔 Agent analyzing: 'Show ward-wise issue distribution as a bar chart'
   📊 Decided to use: get_ward_issue_counts
   ✅ Retrieved 8 data points
   ✅ Generated ECharts configuration
   ✅ Generated successfully!

2. Pie Chart - Status Distribution
   Query: 'Display status breakdown as a pie chart'

🤔 Agent analyzing: 'Display status breakdown as a pie chart'
   📊 Decided to use: get_status_distribution
   ✅ Retrieved 5 data points
   ✅ Generated ECharts configuration
   ✅ Generated successfully!

3. Bar Chart - Severity Levels
   Query: 'Show severity levels as a bar chart'

🤔 Agent analyzing: 'Show severity levels as a bar chart'
   📊 Decided to use: get_severity_breakdown
   ✅ Retrieved 4 data points
   ✅ Generated ECharts configuration
   ✅ Generated successfully!

4. Line Chart - Daily Trends
   Query: 'Create a line chart showing issue trends over time'


In [ ]:
# ============================================
# SAMPLE FRONTEND - FastAPI + HTML
# ============================================

print("🎨 Creating Sample Frontend...")
print("="*60)

# Install pyngrok for Colab environment
!pip install -q pyngrok

from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import threading

🎨 Creating Sample Frontend...


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8001): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2198669357.py", lin

In [ ]:
# ============================================
# SETUP NGROK AUTH (Run this ONCE)
# ============================================

from pyngrok import ngrok

# 👇 Paste your ngrok auth token here
NGROK_AUTH_TOKEN = "37pKhszWZ8lvedNG3br5WGtrRyi_7ybrq3ftgyCB3HqreZgBA"

# Get token from: https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("✅ Ngrok authenticated!")
print("Now run the frontend cell again!")

✅ Ngrok authenticated!
Now run the frontend cell again!


In [ ]:
# ============================================
# CREATE FASTAPI APP FOR FRONTEND
# ============================================

frontend_app = FastAPI(title="Civic Monitor Dashboard")

# Enable CORS
frontend_app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

print("✅ Frontend app created")

✅ Frontend app created


In [ ]:
# ============================================
# HTML FRONTEND CODE
# ============================================

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Civic Monitor AI Dashboard</title>
    <script src="https://cdn.jsdelivr.net/npm/echarts@5.4.3/dist/echarts.min.js"></script>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }

        .container {
            max-width: 1400px;
            margin: 0 auto;
        }

        .header {
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            margin-bottom: 30px;
            text-align: center;
        }

        .header h1 {
            color: #667eea;
            font-size: 36px;
            margin-bottom: 10px;
        }

        .header p {
            color: #666;
            font-size: 16px;
        }

        .controls {
            background: white;
            padding: 25px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            margin-bottom: 30px;
        }

        .controls h2 {
            color: #333;
            margin-bottom: 20px;
            font-size: 24px;
        }

        .query-section {
            margin-bottom: 20px;
        }

        .query-section label {
            display: block;
            color: #555;
            font-weight: 600;
            margin-bottom: 8px;
        }

        .query-section input {
            width: 100%;
            padding: 12px;
            border: 2px solid #e0e0e0;
            border-radius: 8px;
            font-size: 16px;
            transition: border 0.3s;
        }

        .query-section input:focus {
            outline: none;
            border-color: #667eea;
        }

        .button-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-bottom: 20px;
        }

        .btn {
            padding: 12px 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-size: 14px;
            font-weight: 600;
            transition: transform 0.2s, box-shadow 0.2s;
        }

        .btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);
        }

        .btn:active {
            transform: translateY(0);
        }

        .btn-generate {
            grid-column: 1 / -1;
            background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
            font-size: 16px;
            padding: 15px;
        }

        .chart-container {
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            margin-bottom: 30px;
        }

        #chartDiv {
            width: 100%;
            height: 500px;
        }

        .loading {
            text-align: center;
            padding: 100px;
            color: #999;
            font-size: 18px;
        }

        .loading::after {
            content: '...';
            animation: dots 1.5s steps(4, end) infinite;
        }

        @keyframes dots {
            0%, 20% { content: '.'; }
            40% { content: '..'; }
            60%, 100% { content: '...'; }
        }

        .stats-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }

        .stat-card {
            background: white;
            padding: 25px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            border-left: 5px solid;
        }

        .stat-card h3 {
            color: #666;
            font-size: 14px;
            margin-bottom: 10px;
            text-transform: uppercase;
        }

        .stat-card .value {
            font-size: 36px;
            font-weight: bold;
            color: #333;
        }

        .stat-card.blue { border-left-color: #667eea; }
        .stat-card.green { border-left-color: #10b981; }
        .stat-card.orange { border-left-color: #f59e0b; }
        .stat-card.purple { border-left-color: #8b5cf6; }
    </style>
</head>
<body>
    <div class="container">
        <!-- Header -->
        <div class="header">
            <h1>🏙️ Civic Monitor AI Dashboard</h1>
            <p>AI-Powered Civic Issue Analytics for Vadodara</p>
        </div>

        <!-- Stats Cards -->
        <div class="stats-grid" id="statsGrid">
            <div class="stat-card blue">
                <h3>Total Issues</h3>
                <div class="value" id="totalIssues">-</div>
            </div>
            <div class="stat-card orange">
                <h3>Pending</h3>
                <div class="value" id="pendingIssues">-</div>
            </div>
            <div class="stat-card green">
                <h3>In Progress</h3>
                <div class="value" id="inProgressIssues">-</div>
            </div>
            <div class="stat-card purple">
                <h3>Resolved</h3>
                <div class="value" id="resolvedIssues">-</div>
            </div>
        </div>

        <!-- Controls -->
        <div class="controls">
            <h2>📊 Generate Visualizations</h2>

            <div class="query-section">
                <label for="userQuery">What would you like to visualize?</label>
                <input
                    type="text"
                    id="userQuery"
                    placeholder="E.g., Show ward-wise issue distribution"
                    value="Show ward-wise issue distribution as a bar chart"
                >
            </div>

            <div class="button-grid">
                <button class="btn" onclick="setQuery('Show ward-wise issue distribution')">
                    📍 Ward Distribution
                </button>
                <button class="btn" onclick="setQuery('Display status breakdown as pie chart')">
                    📊 Status Breakdown
                </button>
                <button class="btn" onclick="setQuery('Show severity levels')">
                    ⚠️ Severity Analysis
                </button>
                <button class="btn" onclick="setQuery('Show issue types breakdown')">
                    🔧 Issue Types
                </button>
                <button class="btn" onclick="setQuery('Show daily trends over time')">
                    📈 Daily Trends
                </button>
                <button class="btn" onclick="setQuery('Show engineer workload')">
                    👷 Engineer Workload
                </button>

                <button class="btn btn-generate" onclick="generateChart()">
                    ✨ Generate Chart with AI
                </button>
            </div>
        </div>

        <!-- Chart Display -->
        <div class="chart-container">
            <div id="chartDiv">
                <div class="loading">Select a visualization type to begin</div>
            </div>
        </div>
    </div>

    <script>
        const API_URL = window.location.origin;
        let chart = null;

        // Initialize ECharts
        function initChart() {
            const chartDiv = document.getElementById('chartDiv');
            chart = echarts.init(chartDiv);
        }

        // Load stats
        async function loadStats() {
            try {
                const response = await fetch(`${API_URL}/api/stats`);
                const data = await response.json();

                document.getElementById('totalIssues').textContent = data.total_issues;
                document.getElementById('pendingIssues').textContent = data.pending;
                document.getElementById('inProgressIssues').textContent = data.in_progress;
                document.getElementById('resolvedIssues').textContent = data.resolved;
            } catch (error) {
                console.error('Error loading stats:', error);
            }
        }

        // Set query in input
        function setQuery(query) {
            document.getElementById('userQuery').value = query;
        }

        // Generate chart
        async function generateChart() {
            const query = document.getElementById('userQuery').value;

            if (!query.trim()) {
                alert('Please enter a query!');
                return;
            }

            // Show loading
            document.getElementById('chartDiv').innerHTML =
                '<div class="loading">AI is generating your chart</div>';

            try {
                // Call API to generate chart
                const response = await fetch(`${API_URL}/api/generate-chart`, {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ query: query })
                });

                const result = await response.json();

                if (result.success) {
                    // Reinitialize chart
                    initChart();

                    // Set option
                    chart.setOption(result.echarts_config);

                    // Make chart responsive
                    window.addEventListener('resize', () => {
                        chart.resize();
                    });
                } else {
                    document.getElementById('chartDiv').innerHTML =
                        `<div class="loading">Error: ${result.error}</div>`;
                }

            } catch (error) {
                console.error('Error:', error);
                document.getElementById('chartDiv').innerHTML =
                    '<div class="loading">Error generating chart. Please try again.</div>';
            }
        }

        // Initialize on load
        window.onload = () => {
            loadStats();
        };
    </script>
</body>
</html>
"""

In [ ]:
# ============================================
# API ENDPOINTS
# ============================================

@frontend_app.get("/", response_class=HTMLResponse)
async def home():
    """Serve the HTML frontend"""
    return HTML_TEMPLATE

@frontend_app.get("/api/stats")
async def get_stats():
    """Get summary statistics"""
    try:
        # Get status distribution from MCP
        status_data = mcp_server._get_status_distribution()

        stats = {
            "total_issues": sum(item['count'] for item in status_data['data']),
            "pending": next((item['count'] for item in status_data['data'] if item['status'] == 'pending'), 0),
            "in_progress": next((item['count'] for item in status_data['data'] if item['status'] == 'in_progress'), 0),
            "resolved": next((item['count'] for item in status_data['data'] if item['status'] == 'resolved'), 0),
        }

        return stats
    except Exception as e:
        return {"error": str(e)}

@frontend_app.post("/api/generate-chart")
async def generate_chart_api(request: Request):
    """Generate chart using React Agent"""
    try:
        body = await request.json()
        query = body.get('query', '')

        # Use React Agent to generate chart
        echarts_config = react_agent.generate_chart(query)

        return JSONResponse({
            "success": True,
            "echarts_config": echarts_config
        })

    except Exception as e:
        return JSONResponse({
            "success": False,
            "error": str(e)
        })

print("✅ API endpoints created")

✅ API endpoints created


In [ ]:
# ============================================
# START FRONTEND SERVER
# ============================================

print("\n🚀 Starting Frontend Server...")
print("="*60)

nest_asyncio.apply()

# Setup ngrok authentication (if not done)
try:
    if 'NGROK_AUTH_TOKEN' in dir():
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except:
    pass

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8001)
    print(f"\n✅ FRONTEND IS LIVE!")
    print("="*60)
    print(f"\n🌐 Open this URL in your browser:")
    print(f"   {public_url}")
    print("\n📱 Share with your team:")
    print(f"   {public_url}")
    print("\n" + "="*60)
except Exception as e:
    print(f"\n⚠️  Ngrok error: {e}")
    print("   Server will run on localhost:8001")
    print("   Open: http://localhost:8001")

print("\n💡 Features:")
print("   ✨ AI-powered chart generation")
print("   📊 Multiple visualization types")
print("   🎨 Beautiful responsive UI")
print("   📈 Real-time statistics")

print("\n⚠️  Keep this cell running!")
print("="*60)

# Start server in thread
def run_frontend():
    uvicorn.run(frontend_app, host="0.0.0.0", port=8001, log_level="info")

frontend_thread = threading.Thread(target=run_frontend, daemon=True)
frontend_thread.start()

print("\n✅ Frontend server started!")
print("🎉 PROJECT COMPLETE!")

# Keep running
import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped")


🚀 Starting Frontend Server...

✅ FRONTEND IS LIVE!

🌐 Open this URL in your browser:
   NgrokTunnel: "https://nakita-superelated-nonsyllogistically.ngrok-free.dev" -> "http://localhost:8001"

📱 Share with your team:
   NgrokTunnel: "https://nakita-superelated-nonsyllogistically.ngrok-free.dev" -> "http://localhost:8001"


💡 Features:
   ✨ AI-powered chart generation
   📊 Multiple visualization types
   🎨 Beautiful responsive UI
   📈 Real-time statistics

⚠️  Keep this cell running!

✅ Frontend server started!
🎉 PROJECT COMPLETE!


INFO:     Started server process [296]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8001): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "GET /api/stats HTTP/1.1" 200 OK

🤔 Agent analyzing: 'Show ward-wise issue distribution'
   📊 Decided to use: get_ward_issue_counts
   ✅ Retrieved 8 data points
   ✅ Generated ECharts configuration
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "POST /api/generate-chart HTTP/1.1" 200 OK
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "GET /api/stats HTTP/1.1" 200 OK

🤔 Agent analyzing: 'Show severity levels'


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-9' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8001): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2198669357.py", lin

   📊 Decided to use: get_severity_breakdown
   ✅ Retrieved 4 data points
   ✅ Generated ECharts configuration
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "POST /api/generate-chart HTTP/1.1" 200 OK

🤔 Agent analyzing: 'Display status breakdown as pie chart'
   📊 Decided to use: get_status_distribution
   ✅ Retrieved 5 data points
   ✅ Generated ECharts configuration
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "POST /api/generate-chart HTTP/1.1" 200 OK

🤔 Agent analyzing: 'Show ward-wise issue distribution'
   📊 Decided to use: get_ward_issue_counts
   ✅ Retrieved 8 data points
   ✅ Generated ECharts configuration
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "POST /api/generate-chart HTTP/1.1" 200 OK

🤔 Agent analyzing: 'Display status breakdown as pie chart'
   📊 Decided to use: get_status_distribution
   ✅ Retrieved 5 data points
   ✅ Generated ECharts configuration
INFO:     2409:4090:3097:b605:d0e4:3931:9e01:4759:0 - "POST /api/generate-chart HTTP/1.1" 20